In [ ]:
import pandas as pd
import numpy as np

#read 'time between arrival distribution' table
s=input('Enter the file name of "Time Between Arrival Distribution" table: ')
df0=pd.read_csv(s)
df0.columns=['tba','pb','rd']

#create list to map random digits for time between arrival
lst=[]
lst.append((np.nan,np.nan))
for i in range(len(df0)):
    tmp0=df0.iloc[i]['rd'].split('-')
    if i==len(df0)-1:
        tmp0[1]='1'+tmp0[1]
    val=float(df0.iloc[i]['tba'])
    for j in range(int(tmp0[0]),int(tmp0[1])+1):
        if j==1:
            continue;
        lst.append((j,val))
dt0=dict(lst)

#read 'service time distribution' of first resource
s=input('Enter the file name of "Service Time Distribution" table of higher priority resource: ')
df1=pd.read_csv(s)
df1.columns=['st','pb','rd']

#create list to map random digits for service time of first resource
lst=[]
for i in range(len(df1)):
    tmp0=df1.iloc[i]['rd'].split('-')
    if i==len(df1)-1:
        tmp0[1]='1'+tmp0[1]
    val=float(df1.iloc[i]['st'])
    for j in range(int(tmp0[0]),int(tmp0[1])+1):
        lst.append((j,val))
dt1=dict(lst)

#read 'service time distribution' of second resource
s=input('Enter the file name of "Service Time Distribution" table of lower priority resource: ')
df2=pd.read_csv(s)
df2.columns=['st','pb','rd']

#create list to map random digits for service time of second resource
lst=[]
for i in range(len(df2)):
    tmp0=df2.iloc[i]['rd'].split('-')
    if i==len(df2)-1:
        tmp0[1]='1'+tmp0[1]
    val=float(df2.iloc[i]['st'])
    for j in range(int(tmp0[0]),int(tmp0[1])+1):
        lst.append((j,val))
dt2=dict(lst)

#read random digits for time between arrival
s=input('Enter the file name of "Random Digits for Inter Arrival Time": ')
f=open(s,'r+')
s=f.read()
tmp0=s.split(',')
randt=[]
randt.append(np.nan)
for i in tmp0:
    randt.append(int(i))

#read random digits for service time
s=input('Enter the file name of "Random Digits for Service Time": ')
f=open(s,'r+')
s=f.read()
tmp0=s.split(',')
rands=[]
for i in tmp0:
    rands.append(int(i))
    
#mapping service time
st=[]
for i in rands:
    tmp=[]
    tmp.append(dt1[i])
    tmp.append(dt2[i])
    st.append(tmp)

#algorithm
df3=pd.DataFrame()
lst=[i for i in range(1,len(randt)+1)]
df3['CN.']=lst
df3['RDA']=randt
df3['TBA']=randt
lst=[]
lst.append(np.nan)
for i in range(1,len(df3)):
    tmp=df3.iloc[i]['TBA']
    lst.append(dt0[tmp])
df3['TBA']=lst
lst=[df3.iloc[:i+1]['TBA'].sum() for i in range(len(df3))]
df3['CTA']=lst
df3['RDS']=rands
tsbh=[]
sth=[]
tseh=[]
tsbl=[]
stl=[]
tsel=[]
que=[]
busyh=0
busyl=0
dbg=[]
for i in range(len(df3)):
    at=df3.iloc[i]['CTA']
    st0=st[i][0]
    st1=st[i][1]
    if at>=busyh and at>=busyl:
        tsbh.append(at)
        sth.append(st0)
        tseh.append(at+st0)
        tsbl.append(np.nan)
        stl.append(np.nan)
        tsel.append(np.nan)
        que.append(0)
        busyh=at+st0
    elif at>=busyh:
        tsbh.append(at)
        sth.append(st0)
        tseh.append(at+st0)
        tsbl.append(np.nan)
        stl.append(np.nan)
        tsel.append(np.nan)
        que.append(0)
        busyh=at+st0
    elif at>=busyl:
        tsbh.append(np.nan)
        sth.append(np.nan)
        tseh.append(np.nan)
        tsbl.append(at)
        stl.append(st1)
        tsel.append(at+st1)
        que.append(0)
        busyl=at+st1
    elif at<busyh and at<busyl:
        if busyh<=busyl:
            tsbh.append(busyh)
            sth.append(st0)
            tseh.append(busyh+st0)
            tsbl.append(np.nan)
            stl.append(np.nan)
            tsel.append(np.nan)
            que.append(busyh-at)
            busyh=busyh+st0
            dbg.append([i,4])
        elif busyl<busyh:
            tsbh.append(np.nan)
            sth.append(np.nan)
            tseh.append(np.nan)
            tsbl.append(busyl)
            stl.append(st1)
            tsel.append(busyl+st1)
            que.append(busyl-at)
            busyl=busyl+st1
df3['TSB(HP)']=tsbh
df3['ST(HP)']=sth
df3['TSE(HP)']=tseh
df3['TSB(LP)']=tsbl
df3['ST(LP)']=stl
df3['TSE(LP)']=tsel
df3['TQ']=que

print("Single Queue Double Resource Simulation Table")
print(df3)